In [1]:
# Import libraries
import os
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
from pylab import rcParams
import pandas as pd
import warnings
from sklearnex import patch_sklearn, config_context
from sklearn.cluster import DBSCAN
from sklearn.exceptions import ConvergenceWarning
from sklearn import preprocessing as skp
from sklearn import model_selection as sms
from sklearn import feature_selection as skf
from sklearn import linear_model as slm
import numpy as np
import scipy.stats as stats
from IPython.display import HTML
import util as util
import nibabel as nib
import os
import pickle
from torch import nn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
HTML('''<style>.jupyter-matplotlib {background-color: #000;}.widget-label, .jupyter-matplotlib-header{color: #fff;}.jupyter-button {background-color: #333;color: #fff;}</style>''')

In [3]:
# Specify headers
cv_names = {'subjects',
            'Preoperative Off-Medicine',
            'Preoperative On-Medicine',
            'Postoperative Off-Medicine &Off-Stimulation\n',
            'Postoperative  Off-Medicine &On-Stimulation'}
# Load scores
file_dir = '/home/ali/RadDBS-QSM/data/xlxs/updrs_iii_chh_proc.csv'
motor_df = util.filter_data(file_dir,cv_names,filter_data=False)
# Find cases with all required scores
cv_names = motor_df.columns[:-1]
cv_dict = util.filter_cases(motor_df,cv_names)
subs = cv_dict['subjects']
# Load extracted features
npy_dir = '/home/ali/RadDBS-QSM/data/npy/chh/batch/'
phi_dir = '/home/ali/RadDBS-QSM/data/phi/chh/batch/orig/'
roi_path = '/home/ali/RadDBS-QSM/data/xlxs/chh_rois.csv'
n_cases = len(os.listdir(npy_dir+'X/'))
n_rois = 6
n_features = 1595
all_rois = False
Phi_all, X_all, R_all, K_all, ID_all = util.load_featstruct(phi_dir,npy_dir+'X/',npy_dir+'R/',npy_dir+'K/',n_rois,n_features,all_rois)
ids = np.asarray(ID_all).astype(int)
subsc = subs[np.in1d(subs,ids)]
per_change = ((cv_dict['Preoperative Off-Medicine']-cv_dict['Postoperative  Off-Medicine &On-Stimulation'])/cv_dict['Preoperative Off-Medicine'])[np.in1d(subs,ids)]
pre_imp = ((cv_dict['Preoperative Off-Medicine']-cv_dict['Preoperative On-Medicine'])/cv_dict['Preoperative Off-Medicine'])[np.in1d(subs,ids)]
pre_updrs = cv_dict['Preoperative Off-Medicine'][np.in1d(subs,ids)]
X0_gt = np.load('/home/ali/RadDBS-QSM/data/npy/old/X0_gt_chh_rois.npy')
X_all_c = X_all.reshape(n_cases,n_rois,n_features)
K_all_c = K_all.reshape(n_cases,n_rois,n_features)
R_all_c = R_all.reshape(n_cases,n_rois,n_features)

Index(['subjects', 'LCT ', 'Preoperative Off-Medicine',
       'Preoperative On-Medicine',
       'Postoperative  Off-Medicine &Off-Stimulation\n ',
       'Postoperative  Off-Medicine &On-Stimulation'],
      dtype='object')
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '13', '14', '16', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40']
Allocated arrays
Created feature matrix
Created ROI matrix
Created feature label matrix


In [16]:
results_ls = np.zeros_like(per_change)
c = 0
r = 1
for j in np.arange(len(subsc)):
    test_id = subsc[j]
    test_index = subsc == test_id
    train_index = subsc != test_id
    X_train = X_all_c[train_index,:,:]
    X_test = X_all_c[test_index,:,:]
    y_train = per_change[train_index]
    y_test = per_change[test_index]

    y_cat = y_train <= 0.3
    idy = np.where(y_cat==1)
    # Cross validation
    X0_ss0,scaler_ss,X_test_ss0 = util.model_scale(skp.StandardScaler(),
                                                X_train,train_index,X_test,
                                                test_index,pre_updrs,None,None,None,None,None,None,None,None,None,False,False,False)
    # 10
    cvn = 10
    cv_scores = np.zeros((cvn,1))
    rs = 1
    for jj in np.arange(2,cvn):
      # Resample to avoid stratification errors
      while np.sum(y_cat) < cvn:
        np.random.seed(rs)
        idyr = np.random.choice(np.asarray(idy).ravel())
        X0_ss0 = np.append(X0_ss0,X0_ss0[idyr,:].reshape(1,-1),axis=0)
        y_train = np.append(y_train,y_train[idyr])
        y_cat = y_train <= 0.3
        rs = rs+1
        
      skf_g = sms.StratifiedKFold(n_splits=jj,shuffle=True,random_state=0)
      skf_gen = skf_g.split(X0_ss0,y_cat)
      lasso = slm.LassoLarsCV(max_iter=1000,cv=jj,n_jobs=-1,normalize=False,eps=0.1)
      with warnings.catch_warnings() and np.errstate(divide='ignore', invalid='ignore'):
        # Feature selection
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        sel = skf.RFECV(lasso,step=1000,cv=skf_gen)
        X0_sst = sel.fit_transform(X0_ss0,y_train)
        X0_ss = X0_sst
        est_ls = lasso.fit(X0_ss,y_train)
        cv_scores[jj] = est_ls.score(X0_ss,y_train)
        print('LassoCV score for',jj,'is',cv_scores[jj])
        
    with warnings.catch_warnings() and np.errstate(divide='ignore', invalid='ignore'):        
      best_cv = np.argmax(cv_scores)

      # Break any ties
      if np.sum(cv_scores == best_cv) > 1:
        cv_scores_tb = np.zeros((np.sum(cv_scores == best_cv),1))
        for jjj in (cv_scores == cv_scores(best_cv)):
          if jjj > 0:
            skf_g = sms.StratifiedKFold(n_splits=np.arange(2,cvn)[jjj],shuffle=True,random_state=1)
            skf_gen = skf_g.split(X0_ss0,y_cat) 
            X0_sst = sel.fit_transform(X0_ss0,y_train)
            X0_ss = X0_sst
            lasso = slm.LassoLarsCV(max_iter=1000,cv=np.arange(2,cvn)[jjj],n_jobs=-1,normalize=False,eps=0.1)
            est_ls = lasso.fit(X0_ss,y_train)
            cv_scores_tb[jjj] = est_ls.score(X0_ss,y_train)
        best_cv = np.argmax(cv_scores_tb)
      
      # Fit whole dataset with optimal cv
      lasso = slm.LassoLarsCV(max_iter=1000,cv=best_cv,n_jobs=-1,normalize=False,eps=0.1)
      sel = skf.RFECV(lasso,step=1000,cv=best_cv)
      X0_sst = sel.fit_transform(X0_ss0,y_train)
      X_test_sst = sel.transform(X_test_ss0)
      X0_ss = X0_sst
      X_test_ss = X_test_sst
    # LASSO
    with warnings.catch_warnings():
      warnings.filterwarnings("ignore", category=ConvergenceWarning)
      lasso = slm.LassoLarsCV(max_iter=1000,cv=best_cv,n_jobs=-1,normalize=False,eps=0.1)
      est_ls = lasso.fit(X0_ss,y_train)
    results_ls[c] = est_ls.predict(X_test_ss)
    print('Lasso predicts',str(np.round(results_ls[c],4)),
              'for case with',str(np.round(np.repeat(per_change,r)[c],2)),'and selected CV',best_cv)
  
    c=c+1

      

  


LassoCV score for 2 is [0.05466769]
LassoCV score for 3 is [1.]
LassoCV score for 4 is [0.99686309]
LassoCV score for 5 is [0.96362283]
LassoCV score for 6 is [0.999786]
LassoCV score for 7 is [0.99939213]
LassoCV score for 8 is [0.9127096]
LassoCV score for 9 is [0.99746118]
Lasso predicts 0.8089 for case with 0.72 and selected CV 3
LassoCV score for 2 is [1.]
LassoCV score for 3 is [0.9999469]
LassoCV score for 4 is [0.99972838]
LassoCV score for 5 is [0.97591588]
LassoCV score for 6 is [0.99995388]
LassoCV score for 7 is [0.99671874]
LassoCV score for 8 is [0.99879181]
LassoCV score for 9 is [0.99494667]
Lasso predicts 0.4478 for case with 0.47 and selected CV 2
LassoCV score for 2 is [0.96423787]
LassoCV score for 3 is [0.99872952]
LassoCV score for 4 is [0.99996611]
LassoCV score for 5 is [0.96738277]
LassoCV score for 6 is [1.]
LassoCV score for 7 is [1.]
LassoCV score for 8 is [1.]
LassoCV score for 9 is [0.99991694]
Lasso predicts 0.7604 for case with 0.69 and selected CV 8
Las

In [ ]:
results_ls[results_ls>1] = 1
util.eval_prediction(np.vstack((pre_imp,
                               results_ls,
                               )),
                               per_change,
                               ['LCT',
                                'Lasso',
                                ],(15,5))
plt.ylim([0,2])
plt.xlim([0,2])
plt.style.use('default')